In [1]:
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from sklearn.metrics import classification_report
from scipy.spatial import distance as dist

import time
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import dlib
from imutils import face_utils
from imutils.video import VideoStream
import imutils

from threading import Thread

Using Theano backend.


### Locate the face


In [2]:
face_hog = dlib.get_frontal_face_detector()
def detect_hog(img, face_detector):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detect faces in grayscale
    # 1 is the number of times to upsample the image (helps to detect smaller faces)
    rects = face_detector(img, 1)
    return rects

### Locate the eyes

In [3]:
def draw_roi(eye, image):
    x, y, w, h = cv2.boundingRect(np.array([eye]))
    h = w
    y = y - h
    x = x - w // 2
    cv2.rectangle(image, (x, y), (x+(w*2), y+(h*2)), (255,0,0), 2)
    roi = image[y:y + (h*2), x:x + (w*2)]
    roi = imutils.resize(roi, width=24, inter=cv2.INTER_CUBIC)

    return roi

In [4]:
predictor = dlib.shape_predictor('facial-landmarks/shape_predictor_68_face_landmarks.dat')
def crop_eyes(frame):
    # detect the face at grayscale image
    rect = detect_hog(frame, face_hog)
    
    # if the face detector doesn’t detect face
    # return None, else if detects more than one faces
    # keep the bigger and if it is only one keep one dim
    if len(rect) == 0:
        return None
    elif len(rect) > 1:
        face = rect[0]
    elif len(rect) == 1:
        [face] = rect
        
    (f_x, f_y, f_w, f_h) = face_utils.rect_to_bb(face)
    cv2.rectangle(frame, (f_x, f_y), (f_x + f_w, f_y + f_h), (0, 255, 0), 2) 
    
    # determine the facial landmarks for the face region
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)
        
    # grab the indexes of the facial landmarks for the left and right eye, respectively
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
    
    # extract the left and right eye indexes
    left_eye = shape[lStart:lEnd]
    right_eye = shape[rStart:rEnd]
    
    # get ROI (region of interest) for the eyes
    left_eye_image = draw_roi(left_eye, frame)
    right_eye_image = draw_roi(right_eye, frame)
    
    # if it doesn’t detect left or right eye return None
    if 0 in left_eye_image.shape or 0 in right_eye_image.shape:
        return None

    # return left and right eye
    cv2.imwrite('temp/temp.png', left_eye_image)
    return left_eye_image, right_eye_image



In [5]:
def cnn_predict(img, model):
    # convert to greyscale and scale between 0 and 1
    img = np.dot(np.array(img, dtype='float32'), [[0.2989], [0.5870], [0.1140]]) / 255
    # keras needs (row, width, height, channel)
    # expand to add row dimension
    x = np.expand_dims(img, axis=0)
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    return classes[0][0]

In [6]:
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    # load model architecture
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_path)
    # compile the loaded model
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    return model

In [7]:
def sound_alarm(path):
    # play an alarm sound
    playsound.playsound(path)

In [ ]:
def main():
    counter = 0
    alarm_on = False
    max_frame = 20

    model = load_model('model_test.json', 'weight_test.h5')
    
    img = cv2.imread('hog.png')
    img = imutils.resize(img, width=500)
    
    left_eye, right_eye = crop_eyes(img)
    
    state_left = cnn_predict(left_eye, model)
    state_right = cnn_predict(right_eye, model)
    
    if state_left == 0 and state_right == 0:
        cv2.putText(img, "Closed", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
    else:
        cv2.putText(img, "Opened", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    cv2.imshow("Face", img)
    cv2.waitKey(0)
    print(img.shape)
    
    print(state_left, state_right)
        
        

    

In [ ]:
main()

/Users/brendonhapp/anaconda/envs/py36/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
